In [ ]:
import sys

!{sys.executable} -m pip install wordcloud Pillow nltk lxml

In [ ]:
import pandas as pd
import numpy as np
import re
import os
from collections import Counter
import glob
import zipfile
import numpy
import re
import nltk
from nltk.tokenize import RegexpTokenizer

HOME_DIR = "/home_remote"
HOME = "/home/thi.tra.my.nguyen"

In [ ]:
def datacollection(wpattern="./*.zip"):
    df = pd.DataFrame()
    for g in glob.glob(wpattern):
        zf = zipfile.ZipFile(g)
        dfs = [pd.read_xml(zf.open(f)) for f in zf.namelist()[1:]]
        df = pd.concat(dfs, ignore_index = True)

    return df

test = datacollection(os.path.join(HOME_DIR, "eRisk2018training/2017_test/*.zip"))

In [ ]:
def file_concat(path):
    train_neg = pd.DataFrame()
    for i in os.listdir(path):
        #print(i)
        allFiles = glob.glob(path+"/" + i+ "/*.xml")
        dt = []
        for x in allFiles:
            try:
                df = pd.read_xml(x)
                dt.append(df)
                #print(dt)
                train_neg = pd.concat(dt)
            except:
                print("This file error " + x)
    return train_neg

In [ ]:
train_neg = file_concat(os.path.join(HOME_DIR, "eRisk2018training/2017_train/negative_examples_anonymous_chunks"))
train_pos = file_concat(os.path.join(HOME_DIR, "eRisk2018training/2017_train/positive_examples_anonymous_chunks"))

In [ ]:
train_neg

### EDA

#### Word Cloud

In [ ]:
train_pos = train_pos.dropna(subset=['TEXT'])
train_neg = train_neg.dropna(subset=['TEXT'])

In [ ]:
#reference: https://docs.python.org/3/library/re.html and 
#https://thinkinfi.com/fasttext-word-embeddings-python-implementation/
def process_text(document):

        # Remove extra white space from text
        document = re.sub(r'\s+', ' ', document, flags=re.I)
         
        # Remove all the special characters from text
        document = re.sub(r'\W', ' ', str(document))
 
        # Remove all single characters from text
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
 
        # Converting to lowercase
        document = document.lower()
 
        return document

#reference: https://www.nltk.org/api/nltk.tokenize.regexp.html
cleaned_train = [process_text(sentence) for sentence in train_neg['TEXT'] if sentence.strip() !='']
tokenizer_train = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
train_token_neg= [tokenizer_train.tokenize(sent) for sent in cleaned_train]

cleaned_test = [process_text(sentence) for sentence in train_pos['TEXT'] if sentence.strip() !='']
train_token_pos = [tokenizer_train.tokenize(sent) for sent in cleaned_test]


In [ ]:
#flatten the list train_token_neg and train_token_pos
train_token_neg = [item for sublist in train_token_neg for item in sublist]
train_token_pos = [item for sublist in train_token_pos for item in sublist]

In [ ]:
#remove stopwords in train_token_neg and train_token_pos
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
train_token_neg = [w for w in train_token_neg if not w in stop_words]
train_token_pos = [w for w in train_token_pos if not w in stop_words]

#remove duplicates
train_token_neg = list(dict.fromkeys(train_token_neg))
train_token_pos = list(dict.fromkeys(train_token_pos))

#remove words with length less than 3
train_token_neg = [w for w in train_token_neg if len(w) >= 3]
train_token_pos = [w for w in train_token_pos if len(w) >=3]

#remove words are popular with frequency more than 10 times in both positive and negative
freq_neg = pd.Series(' '.join(train_neg['TEXT']).split()).value_counts()[:50]
freq_pos = pd.Series(' '.join(train_pos['TEXT']).split()).value_counts()[:50]
freq_neg = list(freq_neg.index)
freq_pos = list(freq_pos.index)
train_token_neg = [w for w in train_token_neg if w not in freq_pos]
train_token_pos = [w for w in train_token_pos if w not in freq_neg]


train_token_neg = [w for w in train_token_neg if w not in ['college', 'hear','seem','cup']]
train_token_pos = [w for w in train_token_pos if w not in ['call','economic','american','one','law']]

In [ ]:
#wordcloud for train_token_final and test_token_final
import matplotlib.pyplot as plt
from wordcloud import WordCloud

text1 = " ".join(train_token_neg)
text2 = " ".join(train_token_pos)

# Create and generate a word cloud image:
font_path = os.path.join(HOME_DIR, 'fonts/Arial.ttf')
wordcloud_neg= WordCloud(width=1000, height=600, background_color='white', font_path=font_path).generate(text1)
wordcloud_pos= WordCloud(width=1000, height=600, background_color='white', font_path=font_path).generate(text2)
plt.figure(figsize=(10,6))
plt.imshow(wordcloud_neg, interpolation='bilinear')
plt.axis("off")
plt.show()
plt.figure(figsize=(10,6))
plt.imshow(wordcloud_pos, interpolation='bilinear')
plt.axis("off")
plt.show()
#save picture as pdf
wordcloud_neg.to_file(os.path.join(HOME,  'wordcloud_neg.pdf'))
wordcloud_pos.to_file(os.path.join(HOME, 'wordcloud_pos.pdf'))


### Uni, Bi Gram and Inforamtion Gain

In [ ]:
%store -r df
%store -r df_neg
df['Label'] = 1
df_neg['Label'] = 0

: 

In [ ]:
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
df2= df.copy()
def preprocess_text(text):
    tokens = nltk.word_tokenize(text)
    return tokens

df2['tokens'] = df2['Text'].apply(preprocess_text)

# Create bigrams
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(df2['tokens'].tolist())

# Get the top 100 bigrams based on frequency
top_100_bigrams = finder.nbest(bigram_measures.likelihood_ratio, 100)

print(top_100_bigrams)

In [ ]:
df2

In [ ]:
#bi-gram for text in df
from nltk import ngrams
from collections import Counter

def get_ngrams(text, n):
    n_grams = ngrams(nltk.word_tokenize(text), n)
    return [' '.join(grams) for grams in n_grams]
pos_text = [ get_ngrams(text, 2) for text in df['Text']]
neg_text = [ get_ngrams(text, 2) for text in df_neg['Text']]

#remove the bi-grams in pos_text that are frequently appeared in neg_text
pos_text = [item for sublist in pos_text for item in sublist]
neg_text = [item for sublist in neg_text for item in sublist]
pos_text = [w for w in pos_text if w not in neg_text]

In [ ]:
# Create bigrams
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(pos_text)

# Get the top 100 bigrams based on frequency
top_100_bigrams = finder.nbest(bigram_measures.likelihood_ratio, 100)

print(top_100_bigrams)

In [ ]:
pos_text